<a href="https://colab.research.google.com/github/gabrielawad/talleresGoogleColab/blob/main/Taller_19_Redes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## «*Los peces no ven el agua*».
### [Hayashi Tadasu](https://es.wikipedia.org/wiki/Hayashi_Tadasu)

# Taller 19 redes

En este taller se presenten los conceptos básicos relativos a la solución de problemas de redes con CVXPY.



# Algunas características especiales de CVXPY (repaso)

CVXPY contiene algunos métodos especiales que facilitan el trabajo con matrices.

El método **sum** permite sumar los elementos de filas, columnas y de toda la matriz.

El método **multiply** multiplica dos matrices, posición a posición

In [ ]:
# Importar las librerías que se van a utilizar
import numpy as np
import cvxpy as cvx
from cvxopt import solvers

# Definir el arreglo d con dimensión (3,4)
d=cvx.Parameter((3,4))

# Ingresar los valores de d. Cada corchete representa una columna
d.value=[[0,1,2],[3,4,5],[6,7,8],[9,10,11]]

# Mostrar el contenido de los valores de d
print("Los valores contenidos en d son:\n", d.value,"\n")

# Mostrar la fila 0 de d
print("Los valores contenidos en la fila 0 de d son:\n", d[0,:].value, "\n")

# Sumar los valores de una fila
dfila0 = sum(d[0, :].value)

# Mostrar la suma de los elementos de la fila
print ("La suma de los elementos de la fila 0 es:", dfila0, "\n")

# Mostrar la columna 1 de d
print("Los valores contenidos en la columna 1 de d son:\n", d[:,1].value, "\n")

# Sumar los valores de una columna
dcol1 = sum(d[:, 1].value)

# Mostrar la suma de los elementos de una columna
print ("La suma de los elementos de la columna 1 es:", dcol1, "\n")

# Multiplicar el arreglo d por el arreglo d utilizando multiply equivale
# a elevar al cuadrado cada una de las posiciones del arreglo d
g = cvx.multiply (d,d)

# Mostrar el contenido de los valores de g
print("Los valores contenidos en g son:\n", g.value,"\n")

# El problema de asignación como un caso de redes

A continuación se presenta un ejemplo del problema de asignación resuelto como si fuera un problemas de redes.

En vez de una lista con nueve posiciones se va a utilizar una matriz de tres filas (origen = hijos) y tres columnas (destino = tareas).

# Problema de asignación (caso trivial)

Tres hermanos Jaime, Pipe y Linda, quieren ganar algún dinero para cubrir sus gastos de celular del mes actual.
Su padre ha diseñado tres posibles tareas para ayudarle en este tema a sus hijos: lavar a mano los tenis, planchar las camisas de cuello y bañar los tres gatos. El padre le pidió a cada hijo que entregara un papel secreto donde se indicara el pago que cada hijo consideraría justo por cada una de las tareas. Los hermanos se comprometieron a aceptar la asignación de tareas que finalmente haga su padre.

Las cotizaciones presentadas por los hijos fueron:
* Jaime: 15 por lavar tenis, 10 por planchar camisas, 9 por bañar gatos
* Pipe: 9 por lavar tenis, 15 por planchar camisas, 10 por bañar gatos
* Linda: 10 por lavar tenis, 12 por planchar camisas, 8 por bañar gatos

¿Qué tareas debería asignar el padre a cada hijo, con el fin de gastar lo menos posible?
Tenga en cuenta que cada tarea puede ser asignada solamente una vez, y que cada hermano debe tener como mínimo una tarea por realizar.

Los datos se pueden consultar en: [Asignacion-tareas-hogar](https://docs.google.com/spreadsheets/d/1ujsT41Jin8IJZM3GT1cfnj0WB2ORTQ0Nuyv87xyx_Ao/edit?usp=sharing)


## Solución 1

In [ ]:
# Importar las librerías que se van a utilizar
import numpy as np
import cvxpy as cvx

# Crear las variables a optimizar. Las filas corresponden a los hijos
# y las columnas a las tareas
x = cvx.Variable((3,3), boolean =True)

# Crear el arreglo con los coeficientes de costos.
# Cada corchete representa una columna, no una fila
c=cvx.Parameter((3,3))
c.value=[[15, 9, 10],[10, 15, 12],[9, 10, 8]]

print ("Los valores contenidos en c son:\n" , c.value, "\n")

nombres = ["Jaime", "Pipe", "Linda"]

tareas =["lava tenis", "plancha camisas", "baña gatos"]

# Definir la función objetivo utilizando sum y multiply
obj_00=cvx.Minimize(cvx.sum(cvx.multiply(c,x)))

# Establecer las restricciones
restricciones_00 = [
    # Restricciones de Jaime
    cvx.sum(x[0,:])==1,
    # Restricciones de Pipe
    cvx.sum(x[1,:])==1,
    # Restricciones de Linda
    cvx.sum(x[2,:])==1,
    # Restricciones de lavar tenis
    cvx.sum(x[:,0])==1,
    # Restricciones de planchar camisas
    cvx.sum(x[:,1])==1,
    # Restricciones de bañar gatos
    cvx.sum(x[:,2])==1
]

# Configurar el problema
prob_00 = cvx.Problem(obj_00, restricciones_00)

# Obtener el valor óptimo
prob_00.solve()

# Mostrar el estado de la solución y el valor óptimo de la función objetivo
print("Estado de la solución:", prob_00.status,"\n"
      + "El costo total de las tareas será:", "%.0f"% prob_00.value, "\n")

# Copiar los valores óptimos de las variables de decisión en un arreglo de NumPy
newX = np.round(x.value)

# Mostrar las asignaciones
for i in range (0,3):
    for j in range (0,3):
        # Seleccionar los valores de la solución iguales a 1.
        if newX[i,j]== 1:
            # Mostrar nombre, tarea y costo
            print(nombres[i], tareas[j], "y cobra", int(c[i,j].value), "pesos.")

## Solución 2

### Paso 0: importar las librerías

In [ ]:
# Importar las librerías que se van a utilizar
import numpy as np
import pandas as pd
import cvxpy as cvx

### Paso 1: definir las variables

Son nueve (9) variables booleanas, cada una de ellas representando la ejecución, o no, de una tarea por uno de los hermanos.

In [ ]:
# Crear las variables a optimizar. Las filas corresponden a los hermanos
# y las columnas a las tareas
asignaciones = cvx.Variable((3,3), boolean =True)

### Paso 2: crear los coeficientes de costos

Los coeficientes de costos se pueden ver en el enlace: [coeficientes_costos](https://github.com/gabrielawad/talleresGoogleColab/blob/main/Archivo_datos/asignacion-tareas-hogar%20-%20coeficientes_costos.csv).

Se va a utilizar el sistema [UTF-8](https://en.wikipedia.org/wiki/UTF-8#:~:text=UTF%2D8%20is%20the%20dominant,8%20encodings%20on%20the%20web.).

In [ ]:
# Establecer la ruta para acceder a los coeficientes de costos
ruta_coef_costos = "https://raw.githubusercontent.com/gabrielawad/\
TalleresGoogleColab/main/Archivo_datos/asignacion-tareas-hogar%20-\
%20coeficientes_costos.csv"

# Leer el contenido de la ruta
datos_coef_costos = pd.read_csv(ruta_coef_costos, delimiter=',',
                          index_col='nombres', encoding='utf-8')

# Verificar la lectura del contenido de la ruta
print("Los coeficientes de costos son: \n", datos_coef_costos, "\n")

# Crear arreglo de NumPy con los valores numéricos del DataFrame
# print("El tipo de datos es:",type(datos_coef_costos.values), "\n")
print("Valores numéricos de los coeficientes de costos: \n",
      datos_coef_costos.values, "\n")

# Crear el arreglo con los coeficientes de costos en CVXPY
coef_costos = cvx.Parameter((3,3))

# Asignar los valores a los coeficientes de costos
coef_costos.value = datos_coef_costos.values

# Mostrar los valores de los coeficientes de costos en CVXPY
print ("Los valores de los coeficientes de costos en CVXPY son: \n",
       coef_costos.value, "\n")

### Paso 3: crear la función objetivo

In [ ]:
# Definir la función objetivo utilizando sum y multiply
obj_asignar_tareas=cvx.Minimize(cvx.sum(cvx.multiply(coef_costos,asignaciones)))

### Paso 4: leer los coeficientes técnicos

Los coeficientes técnicos se pueden ver en el siguiente enlace: [coeficientes_técnicos](https://github.com/gabrielawad/talleresGoogleColab/blob/main/Archivo_datos/asignacion-tareas-hogar%20-%20coeficientes_técnicos.csv)


In [ ]:
# Establecer la ruta para acceder a los coeficientes técnicos
ruta_coef_tecnicos = "https://raw.githubusercontent.com/gabrielawad/talleres\
GoogleColab/main/Archivo_datos/asignacion-tareas-hogar%20-\
%20coeficientes_t%C3%A9cnicos.csv"

# Leer el contenido de la ruta
datos_coef_tecnicos = pd.read_csv(ruta_coef_tecnicos, delimiter=',',
                            index_col='nombres', encoding='utf-8')

# Verificar la lectura del contenido de la ruta
print("Los coeficientes técnicos son: \n", datos_coef_tecnicos, "\n")

# Crear arreglo de NumPy con los valores numéricos del DataFrame
# print("El tipo de datos es:",type(datos_coef_tecnicos.values), "\n")
print("Valores numéricos de los coeficientes técnicos: \n",
      datos_coef_tecnicos.values, "\n")

# Crear el arreglo los coeficientes técnicos en CVXPY
coef_tecnicos = cvx.Parameter((3,3))

# Asignar los valores a los coeficientes técnicos
coef_tecnicos.value = datos_coef_tecnicos.values

# Mostrar los valores de los coeficientes tecnicos en CVXPY
print ("Los valores de los coeficientes técnicos en CVXPY son: \n",
       coef_tecnicos.value, "\n")

### Paso 5: leer las restricciones

Las restricciones se pueden ver en el siguiente enlace: [restricciones](https://github.com/gabrielawad/talleresGoogleColab/blob/main/Archivo_datos/asignacion-tareas-hogar%20-%20restricciones.csv)

In [ ]:
# Establecer la ruta para acceder a los coeficientes técnicos
ruta_restricciones = "https://raw.githubusercontent.com/gabrielawad/talleres\
GoogleColab/main/Archivo_datos/asignacion-tareas-hogar%20-%20restricciones.csv"

# Leer el contenido de la ruta
datos_restricciones = pd.read_csv(ruta_restricciones, delimiter=',',
                            index_col='restricciones', encoding='utf-8')

# Verificar la lectura del contenido de la ruta
print("Las restricciones son: \n", datos_restricciones, "\n")

# Crear arreglo de NumPy con los valores numéricos del DataFrame
# print("El tipo de datos es:",type(datos_restricciones.values), "\n")
print("Valores numéricos de las restricciones: \n", datos_restricciones.values)

# Crear el arreglo de restricciones en CVXPY
restricciones = cvx.Parameter((6,1))

# Asignar los valores a las restricciones
restricciones.value = datos_restricciones.values

# Mostrar los valores de las restricciones en CVXPY
print ("Los valores de las restricciones en CVXPY son: \n",
       restricciones.value, "\n")

### Paso 6: Establecer las ecuaciones y las inecuaciones que contienen las restricciones

Dadas las características de este ejercicio todas las restricciones son ecuaciones. Se debe asignar una tarea a cada persona y cada tarea debe tener un persona asignada.

In [ ]:
# Crear las restricciones
rest_asignar_tareas = [
  # Restricciones de Jaime
  cvx.sum(cvx.multiply(coef_tecnicos[0,:],asignaciones[0,:]))==restricciones[0],

  # Restricciones de Pipe
  cvx.sum(cvx.multiply(coef_tecnicos[1,:],asignaciones[1,:]))==restricciones[1],

  # Restricciones de Linda
  cvx.sum(cvx.multiply(coef_tecnicos[2,:],asignaciones[2,:]))==restricciones[2],

  # Restricciones de lavar tenis
  cvx.sum(cvx.multiply(coef_tecnicos[:,0],asignaciones[:,0]))==restricciones[3],

  # Restricciones de planchar camisas
  cvx.sum(cvx.multiply(coef_tecnicos[:,1],asignaciones[:,1]))==restricciones[4],

  # Restricciones de bañar gatos
  cvx.sum(cvx.multiply(coef_tecnicos[:,2],asignaciones[:,2]))==restricciones[5]
]

### Paso 7: estructurar el problema y resolver

In [ ]:
# Configurar el problema
prob_asignar_tareas = cvx.Problem(obj_asignar_tareas, rest_asignar_tareas)

# Obtener el valor óptimo
prob_asignar_tareas.solve()

# Mostrar el estado de la solución y el valor óptimo de la función objetivo
print("Estado de la solución:", prob_asignar_tareas.status,"\n")

### Paso 8: mostrar resultados

In [ ]:
# Mostrar el valor óptimo de la función objetivo
print("El costo total de las tareas es: ", int(prob_asignar_tareas.value), "\n")

# Mostrar las asignaciones en CVXPY
print("Asignaciones en CVXPY: \n", asignaciones.value, "\n")

# Crear un DataFrame para mostrar las asignaciones
df_asignaciones = pd.DataFrame(asignaciones.value,
                               index=datos_coef_costos.index,
                               columns=datos_coef_costos.columns,
                               dtype=np.int64)

# Mostrar las asignaciones
print("Estas son las asignaciones:\n", df_asignaciones, "\n")

# Copiar los valores óptimos de las variables de decisión en un arreglo de NumPy
asignaciones_np = np.round(asignaciones.value)

# Copiar los coeficientes de costos en un arreglo de NumPy
coef_costos_np = np.round(coef_costos.value)

# Calcular los costos de las asignaciones
costos_asignaciones_np = np.multiply(coef_costos_np, asignaciones_np)

# Mostrar los costos de las asignaciones
print("Estos son los costos de las asignaciones: \n",
      costos_asignaciones_np, "\n")

# Crear un DataFrame para mostrar los costos de las asignaciones
df_costo_asignaciones = pd.DataFrame(costos_asignaciones_np,
                               index=datos_coef_costos.index,
                               columns=datos_coef_costos.columns,
                               dtype=np.int64)

# Mostrar los costos de las asignaciones
print("Estas son los costos de las asignaciones:\n",
      df_costo_asignaciones, "\n")

### Paso 9: análisis de holguras

In [ ]:
# Crear un DataFrame para mostrar las holguras de las restricciones
analisis_holguras = datos_restricciones.copy()

# Mostrar el estado inicial del DataFrame con las holguras de las restricciones
print("Estas son las restricciones:\n", analisis_holguras, "\n")

# Cambiar el nombre de la columa "valores" por "restricciones"
analisis_holguras.rename(columns={"valores": "restricciones"}, inplace=True)

# Calcular el estado de las restricciones en la solución
# Calcular las restricciones de los hermanos
optimo_restricciones_hnos = np.sum(np.multiply(
    datos_coef_tecnicos.values, asignaciones_np),dtype=np.int64, axis=1)

# Calcular las restricciones de los tareas
optimo_restricciones_tareas = np.sum(np.multiply(
    datos_coef_tecnicos.values, asignaciones_np), dtype=np.int64, axis=0)

# Unir las restricciones
optimos =np.concatenate((optimo_restricciones_hnos, optimo_restricciones_tareas))

# Agregar columna con los valores de las restricciones en el punto óptimo
analisis_holguras["óptimos"] = optimos

# Agregar columna con la diferencia entre las restricciones
# y las restricciones en el punto óptimo
analisis_holguras["diferencias"] = analisis_holguras["restricciones"] \
- analisis_holguras["óptimos"]

# Mostrar el analisis de las holguras de las restricciones
print("Estas son las holguras de las restricciones:\n", analisis_holguras, "\n")

# Ejercicios

Resuelva los tres ejercicios propuestos en el [Taller 08 redes con CVXPY](https://docs.google.com/document/d/1wXouIvohWIGqDl3njbWQn-eLt95i69sJaKxsRK-7QWE/edit?usp=sharing).

## Distribución de producción

## Minimizar tiempo de ejecución

## Empleados de un call center